pandas 객체에 저장된 데이터는 여러 내장 함수를 이용해서 합칠 수 있다.
- `pandas.merge`는 하나 이상의 키를 기준으로 DataFrame의 로우를 합친다. SQL이나 다른 관계형 데이터 베이스의 join 연산과 유사하다.
- `pandas.concat`은 하나의 축을 따라 객체를 이어붙인다.
- `combine_frist` 인스턴스 메서드는 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채울 수 있도록 한다.

`merge`나 `join` 연산은 관계형 데이터베이스의 핵심적인 연산으로, 키를 하나 이상 사용해서 데이터 집합의 로우를 합친다. pandas의 `merge`함수를 이용해서 이런 알고리즘을 데이터에 적용할 수 있다.

In [4]:
import pandas as pd
import numpy as np

In [6]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'],
                'data1':range(7)})

In [8]:
df2 = pd.DataFrame({'key':['a','b','d'],
                'data2':range(3)})

In [16]:
df1.set_index('key')

,data1
key,
b,0
b,1
a,2
c,3
a,4
a,5
b,6


In [17]:
df2.set_index('key')

,data2
key,
a,0
b,1
d,2


In [19]:
pd.merge(df1,df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [18]:
pd.merge(df1,df2).set_index('key')

,data1,data2
key,,
b,0,1
b,1,1
b,6,1
a,2,0
a,4,0
a,5,0


`merge` 함수는 기본적으로 inner join을 수행하여 교집합인 결과를 반환한다. how 인자로 `left`,`right`,`outer`를 넘겨서 조인을 수행할 수도 있다.
`outer` 조인은 합집합 결과를 반환하고 `left`,`right`는 각각 왼쪽 혹은 오른쪽 로우를 포함하는 결과를 반환한다.

In [21]:
pd.merge(df1, df2, how='outer').set_index('key')

,data1,data2
key,,
b,0.0,1.0
b,1.0,1.0
b,6.0,1.0
a,2.0,0.0
a,4.0,0.0
a,5.0,0.0
c,3.0,NaN
d,NaN,2.0


#### 여러 개의 키를 병합하려면 컬럼 이름이 들어간 리스트를 넘기면 된다.

In [22]:
left = pd.DataFrame({'key1':['foo','foo','bar'],
                    'key2':['one','two','one'],
                    'lval':[1,2,3]})

In [23]:
right = pd.DataFrame({'key1':['foo','foo','bar','bar'],
                     'key2':['one','one','one','two'],
                     'rval':[4,5,6,7]})

In [24]:
pd.merge(left,right)

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,bar,one,3,6


In [27]:
pd.merge(left,right, on=['key1','key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


`merge` 메서드의 종류에 따라 어떤 키 조합이 결과로 반환되는지 알려면 실제 구현과는 조금 다르지만<br/>
여러 개의 키가 들어있는 튜플의 배열이 단일 조인 키로 사용된다고 생각하면 된다.
#### 컬럼과 컬럼을 조인할 때 전달한 DataFrame 객체의 색인은 무시된다.

merge 연산에서 고려해야 할 마지막 사항은 겹치는 컬럼 이름에 대한 처리다.<br/>
축의 이름을 변경해서 수동으로 컬럼 이름을 겹치게 할 수도 있고, merge 함수에 있는 suffixes 인자를 통해<br/>
두 DataFrame 객체에서 겹치는 컬럼 이름 뒤에 붙일 문자열을 지정해 줄 수도 있다.

In [28]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [32]:
pd.merge(left, right, on='key1', suffixes=('_left','_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7
